# Lesson 2 - Assignment

In this assignment, you will implement a Decision Tree algorithm from scratch and compare the results to existing sklearn algorithm. 

In [1]:
# import packages
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from matplotlib.legend_handler import HandlerLine2D
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV

# make this notebook's output stable across runs
np.random.seed(0)

## NOTE

### I decided to use pandas instead of a list to represent the data, just because I found it easier to understand what was actually going on that way than when it was represented as just a list of lists.

In [3]:
fn = 'data_banknote_authentication.csv'
df = pd.read_csv(fn, header=None)
df.columns = ['A', 'B', 'C', 'D', 'target']
df

,A,B,C,D,target
0,3.62160,8.66610,-2.8073,-0.44699,0
1,4.54590,8.16740,-2.4586,-1.46210,0
2,3.86600,-2.63830,1.9242,0.10645,0
3,3.45660,9.52280,-4.0112,-3.59440,0
4,0.32924,-4.45520,4.5718,-0.98880,0
...,...,...,...,...,...
1367,0.40614,1.34920,-1.4501,-0.55949,1
1368,-1.38870,-4.87730,6.4774,0.34179,1
1369,-3.75030,-13.45860,17.5932,-2.77710,1
1370,-3.56370,-8.38270,12.3930,-1.28230,1


Question 1.1: Implement the functions to calculate Gini Index.

In [4]:
# Calculate the Gini index for a split dataset
def gini_index(groups, classes, target_col='target'):
    total_instances = float(sum([len(group) for group in groups]))
    gini = 0.0
    for group in groups:
        size = float(len(group))
        if size == 0:
            continue
        score = 0.0
        for class_val in classes:
            p = (group[target_col] == class_val).sum() / size
            score += p**2
        gini += (1 - score) * (size / total_instances)
    return gini

Question 2.1: Write a method that splits the 

In [5]:
# pandas version
# Split a dataset based on an attribute and an attribute value
def test_split(df, feature, value):
    left = df[df[feature] < value]
    right = df[df[feature] >= value]
    return left, right

Question 2.2: Write a method that loops over the dataset, determine the groups of rows that belong to the right or left split, and calculates the gini_index

In [6]:
# pandas version
def get_split(df, target_col='target'):
    class_values = df[target_col].unique()
    best_feature, best_value, best_score, best_groups = 999, 999, 999, None
    for feature in df.columns:
        if feature == target_col:
            continue
        for index in df.index:
            value = df.loc[index][feature]
            groups = test_split(df, feature, value)
            gini = gini_index(groups, class_values)
            if gini < best_score:
                best_feature, best_value, best_score, best_groups = feature, value, gini, groups
    return {
        'feature': best_feature,
        'value': best_value,
        'groups': best_groups
    }
            

Question 2.3: Repeat question 2.2 using entropy

In [ ]:
def entropy_score(groups, classes, target_col='target'):
    total_instances = float(sum([len(group) for group in groups]))
    entropy = 0.0
    for group in groups:
        size = float(len(group))
        if size == 0:
            continue
        score = 0.0
        for class_val in classes:
            p = (group[target_col] == class_val).sum() / size
            score += p * np.log(p)
        entropy += -score * (size / total_instances)
    return entropy

In [ ]:
def get_split_entropy(dataset):
    class_values = df[target_col].unique()
    best_feature, best_value, best_score, best_groups = 999, 999, 999, None
    for feature in df.columns:
        if feature == target_col:
            continue
        for index in df.index:
            value = df.loc[index][feature]
            groups = test_split(df, feature, value)
            entropy = entropy_score(groups, class_values)
            if entropy < best_score:
                best_feature, best_value, best_score, best_groups = feature, value, entropy, groups
    return {
        'feature': best_feature,
        'value': best_value,
        'groups': best_groups
    }

Question 3.1: Write a method that takes in a group of rows and determines the class they belongs to. It should return the most common output value for a list of rows.

In [7]:
# pandas version
def to_terminal(group, target_col='target'):
    return group[target_col].mode()

Question 3.2: Write a method that recursively split the data.
The method takes in a node, max_depth, min_size, and depth. Initially, the method would be called by passing the rood node and depth of 1. Here are the steps to help you implement:

- First, we create two groups for the data split and delete any existing groups from the node. As rows are used, they are no longer needed.
- Second, check if rows should be in left or right group, and if so we create a terminal node using the records we have.
- Third, check if maximum depth is reached and if so we create a terminal node.
- Fourth, process left child, creating a terminal node if the group of rows is too small, otherwise creating and adding the left node in a depth first fashion until the bottom of the tree is reached on this branch.
- Fifth, process the right side in a similar manner as left side, as we rise back up the constructed tree to the root.

In [8]:
# pandas version
def split(node, max_depth, min_size, depth):
    left, right = node['groups']
    del(node['groups'])
    if left.empty or right.empty:
        terminal_val = to_terminal(pd.concat([left, right]))
        node['left'] = terminal_val
        node['right'] = terminal_val
        return
    if depth >= max_depth:
        node['right'] = to_terminal['right']
        node['left'] = to_terminal['left']
        return
    if len(left) <= min_size:
        node['left'] = to_terminal(left)
    else:
        node['left'] = get_split(left)
        split(node['left'], max_depth, min_size, depth+1)
    if len(right) <= min_size:
        node['right'] = to_terminal(right)
    else:
        node['right'] = get_split(right)
        split(node['right'], max_depth, min_size, depth+1)

Question 3.3: Write a method that builds the tree. The method creates an initial split to determine root node, and then calls the split method.

In [9]:
# Build a decision tree
def build_tree(train, max_depth, min_size):
    "TODO get the first split, and then split starting fromt the root"
    root = get_split(train)
    split(root, max_depth, min_size, 1)
    return root

Question 3.4: Write a method that takes in a node and rows of data, and predicts the class associated with each row.

In [10]:
# pandas version
# Make a prediction with a decision tree
def predict(node, row):
    #TODO check if a row belongs to a node and recursively traverse the tree if the row doesn't.
    if row[node['feature']] < node['value']:
        if type(node['left']) is dict:
            return predict(node['left'], row)
        else:
            return node['left']
    else:
        if type(node['right']) is dict:
            return predict(node['right'], row)
        else:
            return node['right']

Question 4: Train a decision tree using the banknote_authentication data

In [11]:
# pandas version
from random import seed
from random import randrange
from csv import reader
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Load a CSV file
def load_csv(filename):
    df = pd.read_csv(filename, header=None)
    df.columns = ['A', 'B', 'C', 'D', 'target']
    return df
       
filename = 'data_banknote_authentication.csv'
df = load_csv(filename)

# the data is sorted by class so this version will have a test set that contains only positive samples
# train = df[1:int(len(df)*2/3)]
# test = df[int(len(df)*2/3)+1:len(df)]

# i'll use train_test_split instead
train, test = train_test_split(df, test_size=2/3, random_state=99)

#### Note: the below might be a little slower than normal, due to the overhead of iterrows(). But it does work! It took about ~12 seconds when I tried it.

In [16]:
#TODO Build a tree and evalute accuracy
tree = build_tree(train, max_depth=100, min_size=1)
predictions = list()
for index, row in test.iterrows():
    prediction = predict(tree, row)
    predictions.append(prediction)
               
print('Accuracy: %s' % accuracy_score(test['target'], predictions))

Accuracy: 0.9508196721311475


[Bonus] Question 5: Train and evaluate an SKLEARN decision tree model, and compare the results to your model 

In [14]:
from sklearn.tree import DecisionTreeClassifier
features = ['A', 'B', 'C', 'D']
clf = DecisionTreeClassifier().fit(train[features], train['target'])
predictions = clf.predict(test[features])

print('Accuracy: %s' % accuracy_score(test['target'], predictions))

Accuracy: 0.9628415300546448


Question 6: Create a new text cell in your Notebook: Complete a 50-100 word summary (or short description of your thinking in applying this week's learning to the solution) of your experience in this assignment. Include: What was your incoming experience with this model, if any? what steps you took, what obstacles you encountered. how you link this exercise to real-world, machine learning problem-solving. (What steps were missing? What else do you need to learn?) This summary allows your instructor to know how you are doing and allot points for your effort in thinking and planning, and making connections to real-world work.

Prior to this assignment, I had no experience with decision trees, so this is truly new material for me. I decided to use pandas to work with the data instead of a raw list of lists. I'm not sure if that was slower; it's possible it was since I mostly used the framework already provided, rather than really taking advantage of pandas vector methods. I'm sure this added some overhead since doing things like iterrows() is really slow in pandas. But it was helpful because then I had a better grasp of what the data was doing at each step, since I am used to working with pandas. 

